# Các thao tác trên data

Sau khi đã đọc data, cần làm sạch và biến đổi data cho phù hợp với mục đích sử dụng. `DataFrame` của pandas là 1 công cụ rất tốt để thực hiện việc này.
Có thể nghĩ đơn giản về DataFrame như 1 spreadsheet. Nó cung cấp các chức năng:
- Lưu trữ và biểu diễn data dưới dạng bảng
- Thêm, xóa, sửa hàng, cột
- Xoay bảng, ghép cột, ghép bảng
- Vẽ đồ thị
- ...

In [ ]:
import pandas as pd

pd.set_option('max.rows', 6)

data = pd.read_csv('data/titanic.csv')
data.index

In [ ]:
data.columns

In [ ]:
data.head()

pandas tự động tạo thêm 1 cột index để dễ dàng quản lý data. Tuy nhiên với bộ data này ta đã có sẵn cột `PassengerId` có thể dùng làm index

In [ ]:
data.set_index('PassengerId', inplace=True)
# data = data.set_index('PassengerId') cho kết quả tương tự
data.head()

## Truy xuất dữ liệu

Để tách lấy 1 cột trong `DataFrame`, có thể dùng _attribute lookup_

In [ ]:
data.Survived

hoặc dùng kiểu truy xuất theo key _getitem_ như với dict

In [ ]:
data['Name']

2 cách trên trả ra object với kiểu dữ liệu là `Series`. Nếu vẫn muốn kiểu trả ra là `DataFrame`, có thể truyền 1 list vào làm index cho _getitem_

In [ ]:
name_series = data['Name']
print(type(name_series))
name_df = data[['Name']]
print(type(name_df))

Có thể dùng cách này để select 1 lúc nhiều cột

In [ ]:
data[['Name', 'Survived']]

Để select data theo hàng, dùng `loc` và `iloc`.
`loc` select data theo __giá trị__ của index

In [ ]:
data.loc[1]

`iloc` select data theo __thứ tự__ (từ 0)

In [ ]:
data.iloc[1]

Cả 2 đều support sử dụng __slice__ để lấy dữ liệu

In [ ]:
data.loc[2:16]

Có thể kết hợp select hàng và cột

In [ ]:
data.iloc[:3, [1, 2]]  # Chọn 3 row đầu, và chỉ lấy cột 1, 2 

In [ ]:
data.loc[[1, 3, 5], ['Name', 'Survived']]

### Excersise

Có 1 số cách khác để select dữ liệu từ `DataFrame`. Thử tìm cách select
- Tất cả các hàng có `PassengerId` lẻ
- Tất cả các cột trừ cột `Survived`

In [ ]:
# Code vào đây

In [ ]:
%load solutions/select_data.py

## Biến đổi dữ liệu

Trong hầu hết các trường hợp, raw data rất loạn và cần được chuẩn hóa. Để biết được cần chuẩn hóa cột nào, phải nhìn vào data thực

In [ ]:
data.info()

Nhìn vào đây ta có thể thấy:
- Có 891 hàng và 11 cột
- Cột `Age` có khoảng 280 ô không có dữ liệu (null)
- Cột `Cabin` có rất ít dữ liệu
- Cột `Embarked` cũng có 2 ô bị null

Ngoài ra cột `Ticket` có vẻ không có nhiều ý nghĩa lắm; số vé thì ảnh hưởng gì đến sống hay chết.

Với các cột thiếu dữ liệu, có nhiều cách đối phó:
- Nếu ít hàng thiếu thì có thể loại bỏ các hàng đó. VD cột `Embarked`
- Nếu quá nhiều hàng thiếu thì có thể vứt bỏ luôn cả cột. VD cột `Cabin`
- Hoặc có thể tìm cách đoán và điền dữ liệu thiếu. Trường hợp này cột `Age` là 1 nhân tố quan trọng trong khả năng sống sót, khó có thể vứt bỏ, nên cần tìm cách điền thêm vào.

In [ ]:
# Loại bỏ cột Cabin, Ticket
data.drop(columns=['Cabin', 'Ticket'], inplace=True)
# Hoặc có thể dùng
# del data['Cabin']
# del data['Ticket']
data.info()

In [ ]:
# Điền thêm thông tin vào cột Age
# Có nhiều cách:
#    Dự đoán dựa vào các thông tin khác
#    Dùng giá trị có tần suất cao nhất
#    Dùng giá trị trung bình
#    Dùng giá trị median
# Ở đây chọn cách dùng median
data['Age'].fillna(data['Age'].median(), inplace=True)
data.info()

Về cột `Embarked`, có thể dễ dàng drop 2 hàng thiếu dữ liệu. Tuy nhiên có 1 câu hỏi: Có cần cột này không?
Cụ thể hơn, việc 1 người lên tàu ở cảng nào có ảnh hưởng đến tỉ lệ sống sót của người đó không?

### Exercise

Nếu câu trả lời cho câu hỏi trên là "Có" thì viết code để loại bỏ 2 hàng thiếu cột `Embarked`.
Hint: dùng `DataFrame.drop` và `DataFrame.index`

Nếu câu trả lời là "Không" thì viết code để loại bỏ cả cột `Embarked`.

Hoặc có thể chọn viết code để input giá trị vào 2 ô thiếu này.

In [ ]:
# Code vào đây

In [ ]:
%load solutions/remove_embarked.py

Ngoài ra, cột `Age` đang có kiểu dữ liệu là `float64`. Có lẽ nên chuyển thành `int64` thì hợp lý hơn.

In [ ]:
data['Age'] = data['Age'].astype(int)
data.info()

Với cột mang tính category, vd `Sex`, `Embarked` và `Pclass`, pandas có kiểu dữ liệu `category` tương ứng. Note là đây là kiểu đặc biệt của pandas chứ không phải của Python

In [ ]:
data['Sex'] = data['Sex'].astype('category')
data['Embarked'] = data['Embarked'].astype('category')
data['Pclass'] = data['Pclass'].astype('category')
data.info()

Có thể thấy memory usage giảm xuống đáng kể sau khi chuyển data sang kiểu `category`.

## Làm đẹp code

Đến đây thì ta đã khá hài lòng với input data rồi.

In [ ]:
# Đây là code từ đầu đến giờ
import pandas as pd

pd.set_option('max.rows', 6)

data = pd.read_csv('data/titanic.csv')
data.set_index('PassengerId', inplace=True)
data.drop(columns=['Cabin', 'Ticket'], inplace=True)
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

data['Age'] = data['Age'].astype(int)
data['Sex'] = data['Sex'].astype('category')
data['Embarked'] = data['Embarked'].astype('category')
data['Pclass'] = data['Pclass'].astype('category')

Trông code hiện tại thì cũng tàm tạm, nhưng với những bộ data khác thì để làm sạch được data cần rất nhiều code rối rắm.

Thử nhìn lại method `pd.read_csv`

In [ ]:
pd.read_csv?

Dùng những parameter này, ta có thể thu gọn code lại rất nhiều. Ví dụ:

In [ ]:
data = pd.read_csv(
    'data/titanic.csv',
    index_col='PassengerId',
    dtype={
        'Sex': 'category'
    }
)
data.info()

### Exercise

Dùng 1 lần `read_csv` để
- Đọc data từ file `data/titanic.csv`
- Chuyển cột `PassengerId` thành cột index
- Loại bỏ các cột không cần thiết `Cabin`, `Ticket`
- Convert kiểu dữ liệu của các cột `Sex`, `Embarked` và `Pclass` thành kiểu `category`

Hint: dùng các parameter `index_col`, `usecols`, `dtype`.

Note: riêng cột `Age` do có missing value nên hơi khó để convert sang `int64` ngay được.

In [ ]:
# Code vào đây

In [ ]:
%load solutions/read_and_modify.py